# Advent of Code - 2025 - Day 10 - Problem 1

https://adventofcode.com/2025/day/10

## Load Source Data

Load source data into `DATA`.

In [11]:
# Read the input file containing turn instructions
with open("data/sampleDay10.txt") as f:
    DATA = [line.strip() for line in f]

# DATA

## Define Machine class

Defines the state of a machine.

buttons: [[3], [1, 3], [2], [2, 3], [0, 2], [0, 1]]
valid_joltages: [3, 5, 4, 7]

0P0 + 0P1 + 0P2 + 0P3 + 1P4 + 1P5 = 3
0P0 + 1P1 + 0P2 + 0P3 + 0P4 + 1P5 = 5
0P0 + 0P1 + 1P2 + 1P3 + 1P4 + 0P5 = 4
1P0 + 1P1 + 0P2 + 1P3 + 0P4 + 0P5 = 7

In [12]:
from constraint.problem import Problem
from constraint.constraints import ExactSumConstraint
import re


class Machine:

    def __init__(self, definition: str):

        valid_lights, buttons, valid_joltages = Machine.parse_definition(definition)

        self.valid_lights = valid_lights
        self.buttons = buttons
        self.valid_joltages = valid_joltages

    @staticmethod
    def create_patterns() -> tuple[str, str]:

        ws = r"\s+"
        light_pattern = r"\[([.#]+)\]"
        button_pattern = r"\(([0-9,]+)\)"
        joltage_pattern = r"\{([0-9,]+)\}"

        # Regular expression pattern for the definition string
        line_pattern = (
            f"{light_pattern}((?:{ws}{button_pattern})+){ws}{joltage_pattern}"
        )

        # Regular expression for individual button definitions
        button_pattern = f"(?:{ws}({button_pattern}))"

        return (line_pattern, button_pattern)

    @staticmethod
    def parse_definition(
        definition: str,
    ) -> tuple[list[bool], list[list[int]], list[int]]:

        # Parse the overall definition string. Note that there are two results returned for the buttons due to the nested grouping constructs: the entire
        # button string as well as the last matched button.
        #
        definition_match = re.match(Machine.line_pattern, definition)
        assert definition_match
        buttons = definition_match.group(2)
        button_matches = re.findall(Machine.button_pattern, buttons)

        # Parse and convert the separate machine attributes.
        #
        valid_lights: list[bool] = [light == "#" for light in definition_match.group(1)]
        buttons = [
            list(map(int, button_match[1].split(",")))
            for button_match in button_matches
        ]
        joltages = list(map(int, definition_match.group(4).split(",")))

        return (valid_lights, buttons, joltages)
    
    def get_coeffcients(self) -> list[list[int]]:
    
        # Create a series of linear equations that express joltages in terms of the corresponding buttons.
        result = [[0 for _ in range(len(self.buttons))] for _ in range(len(self.valid_joltages))]
        for idx_button, button in enumerate(self.buttons):
            # print(f"button = {idx_button} - {button}")
            for idx_joltage in button:
                # print(f"  joltagle = {idx_joltage}")
                result[idx_joltage][idx_button] = 1

        return result
    
    def get_solutions(self):

        c = self.get_coeffcients()
        j = self.valid_joltages

        max_joltage = max(j)

        problem = Problem()
        problem.addVariables(range(len(self.buttons)), range(max_joltage + 1))
        for index, joltage in enumerate(self.valid_joltages):
            problem.addConstraint(ExactSumConstraint(joltage, c[index]))

        solutions = problem.getSolutions()

        return solutions
    
    def get_min_button_presses(self):
        solutions = self.get_solutions()
        min_button_presses = min(sum(value for value in solution.values()) for solution in solutions)
        return min_button_presses

    
    # def get_coeffcients2(self) -> list[list[int]]:
    
    #     result = [[0 for _ in range(len(self.buttons)+1)] for _ in range(len(self.valid_joltages))]
    #     for idx_button, button in enumerate(self.buttons):
    #         # print(f"button = {idx_button} - {button}")
    #         for idx_joltage in button:
    #             # print(f"  joltagle = {idx_joltage}")
    #             result[idx_joltage][idx_button] = 1
    #     for idx_joltage, joltage in enumerate(self.valid_joltages):
    #         result[idx_joltage][len(self.buttons)] = joltage

    #     return result
    
    line_pattern, button_pattern = create_patterns()

## Solve All Machines

buttons: [[3], [1, 3], [2], [2, 3], [0, 2], [0, 1]]
valid_joltages: [3, 5, 4, 7]

0P0 + 0P1 + 0P2 + 0P3 + 1P4 + 1P5 = 3
0P0 + 1P1 + 0P2 + 0P3 + 0P4 + 1P5 = 5
0P0 + 0P1 + 1P2 + 1P3 + 1P4 + 0P5 = 4
1P0 + 1P1 + 0P2 + 1P3 + 0P4 + 0P5 = 7


In [13]:
import numpy as np
from sympy import Matrix, symbols, linsolve, solve
from  constraint.problem import Problem
from constraint.constraints import ExactSumConstraint

# total_presses = 0
# import math

# for line in DATA:
m = Machine(DATA[0])
for attr, value in vars(m).items():
    print(f"{attr}: {value}")    

min_button_presses = m.get_min_button_presses()
min_button_presses
# solutions = m.get_solutions()
# min_button_presses = min(sum(value for value in solution.values()) for solution in solutions)
# min_button_presses

# c = m.get_coeffcients()
# j = m.valid_joltages
# print(c)
# print(j)

# problem = Problem()
# problem.addVariables(list(range(0, 6)), list(range(0, 100)))
# problem.addConstraint(ExactSumConstraint(j[0], c[0]))
# problem.addConstraint(ExactSumConstraint(j[1], c[1]))
# problem.addConstraint(ExactSumConstraint(j[2], c[2]))
# problem.addConstraint(ExactSumConstraint(j[3], c[3]))
# solutions = problem.getSolutions()
# solutions

# print(m.get_joltages([2]))
# A  = np.array(m.get_coeffcients())
# A = Matrix(m.get_coeffcients2())
# # a,b,c,d,e,f = symbols('a,b,c,d,e,f')
# solution = linsolve(A)
# A, solution
# A
# A
# A
# A = Matrix(m.get_coeffcients())
# print(A)
# # B = np.array(m.valid_joltages)
# B = Matrix([3,5,4,7]) # m.valid_joltages)
# A.LUsolve(B)
# # print(B)
# # linsolve
# # X = np.linalg.lstsq(A, B)
# solution = A.LUsolve(B)

# print(solution)
# total_presses += sum(m.get_best_light_solutions())

# print(f"total_presses = {total_presses}")

# m = Machine(DATA[5])
# for attr, value in vars(m).items():
#     print(f"{attr}: {value}")    
# print(m.get_maximum_button_presses())
# print(math.prod(m.get_maximum_button_presses()))


valid_lights: [False, True, True, False]
buttons: [[3], [1, 3], [2], [2, 3], [0, 2], [0, 1]]
valid_joltages: [3, 5, 4, 7]


10